In [2]:
# !pip install unidecode matplotlib pandas

Defaulting to user installation because normal site-packages is not writeable


In [1]:
import os
from pathlib import Path
try: 
    if SCRIPTS_DIR is None:
        SCRIPTS_DIR = Path(os.getcwd())
        PROJECT_DIR = SCRIPTS_DIR/"../../"
        ANALYSIS_DIR = PROJECT_DIR/"build/analysis"
        ARTIFACTS_DIR = ANALYSIS_DIR/"artifacts"
        ANALYSIS_FILE = ANALYSIS_DIR/"analysis.circom"
    
except:
    SCRIPTS_DIR = Path(os.getcwd())
    PROJECT_DIR = SCRIPTS_DIR/"../../"
    ANALYSIS_DIR = PROJECT_DIR/"build/analysis"
    ARTIFACTS_DIR = ANALYSIS_DIR/"artifacts"
    CIRCUITS_DIR = PROJECT_DIR/"circuits/analysis"
    ANALYSIS_FILE = CIRCUITS_DIR/"analysis.circom"
    
os.chdir(PROJECT_DIR)

In [3]:
import subprocess
import shutil
import unidecode

In [4]:
os.makedirs(ANALYSIS_DIR, exist_ok=True)
os.makedirs(ARTIFACTS_DIR, exist_ok=True)

# Compilation constraint check
## Set Membership

### Function definitions

In [71]:
import re
import json
import pandas as pd

# 7-bit C1 ANSI sequences
ansi_escape = re.compile(r'''
    \x1B  # ESC
    (?:   # 7-bit C1 Fe (except CSI)
        [@-Z\\-_]
    |     # or [ for CSI, followed by a control sequence
        \[
        [0-?]*  # Parameter bytes
        [ -/]*  # Intermediate bytes
        [@-~]   # Final byte
    )
''', re.VERBOSE)

def parse_circom_result(raw: str) -> dict:
    raw = unidecode.unidecode(raw).encode('ascii').decode('ascii')
    raw = ansi_escape.sub('', raw) # Cleaning up ansi sequences
    lines = raw.splitlines()[:-4] # Ignoring useless lines
    
    result = {}
    for line in lines:
        key, val = line.split(':')
        result[key] = val
    
    return result

def compile_circuit(circuit_path: str) -> str:
    command = ["circom", "--r1cs", "--wasm", "--sym", "-o", ARTIFACTS_DIR.__str__(), circuit_path]
#     print(command)
    result = subprocess.run(command, capture_output=True)
    if result.stderr != b'':
        raise Exception(result.stderr)
#     if not raw.splitlines()[-1] == "Everything went okay, circom safe":
#         raise Exception("Compilation problem")
    return result.stdout.decode("ascii")

def get_compilation_data(circuit_path: str, length: int) -> pd.DataFrame:
    out = compile_circuit(circuit_path)
    print(f'parsing compilation for {length=}')
    parsed = parse_circom_result(out)
    df = pd.DataFrame(parsed, columns=parsed.keys(), index=[length], dtype=int)
    df['length'] = length
    return df

def get_baseline():
    baseline = 'circuits/analysis/baseline.circom'
    df = get_compilation_data(baseline, 0)
    return df

def write_line(file, line):
    if line.endswith('\n'):
        file.write(line)
    else:
        file.write(f'{line}\n')
    
def generate_new_file(filename: str, length: int):
    with open(filename, "r") as file:
        file_lines = file.readlines()
        base_file = file_lines[:-1]
        relevant_line = file_lines[-1]
        if relevant_line.endswith('\n'):
            relevant_line = relevant_line[:-1]
        
    changed_line = ' '.join(relevant_line.split()[:-1]) + f' {length});'
    
    with open(filename, "w") as file:
        for line in base_file:
            write_line(file, line)
        write_line(file, changed_line)
#         file.write(f'{changed_line}\n')

### Generating DataFrame

In [77]:
shutil.copy("./circuits/test/poseidon_vanchor_2_2.circom", ANALYSIS_FILE) 
        
# set_lengths = [1, 2, 3, 4, 5, 10, 20, 30, 40, 50, 75, 100]
set_lengths = [1]
df = get_baseline()
for set_length in set_lengths:
    change_set_length(ANALYSIS_FILE, set_length)
    df = pd.concat([df, get_compilation_data(ANALYSIS_FILE, set_length)])
    start = time.time()
    generate_phase2(set_length)
    end = time.time()
    took = end-start
    df.loc[df["length"]==set_length, "phase_2_compile_time"] = took
    print(f'TOOK {took:.2f}s TO COMPILE PHASE 2 FOR {set_length}')
    

df.head()

parsing compilation for length=0
parsing compilation for length=1
CompletedProcess(args=['sh', 'scripts/bash/groth16/analysis/gen_phase2_analysis.sh', '1'], returncode=0, stdout=b'./build/analysis/artifacts/1\nSetting up Phase 2 ceremony for analysis\nOutputting circuit_final.zkey and verifier.sol to ./build/analysis/artifacts/1\n\x1b[INFO]  \x1bsnarkJS\x1b: Reading r1cs\n\x1b[INFO]  \x1bsnarkJS\x1b: Reading tauG1\n\x1b[INFO]  \x1bsnarkJS\x1b: Reading tauG2\n\x1b[INFO]  \x1bsnarkJS\x1b: Reading alphatauG1\n\x1b[INFO]  \x1bsnarkJS\x1b: Reading betatauG1\n\x1b[INFO]  \x1bsnarkJS\x1b: Circuit hash: \n\t\td144fa51 d7843885 a93b860c 30e6a994\n\t\t0337bc86 a652ff13 c5169be8 d354eafd\n\t\t30b39e67 ebe68407 2d5a0d20 d9be7ae5\n\t\t965d826a 4cb2b357 db90d976 bb0d0c81\nEnter a random text. (Entropy): \x1b[DEBUG] \x1bsnarkJS\x1b: Applying key: L Section: 0/17745\n\x1b[DEBUG] \x1bsnarkJS\x1b: Applying key: H Section: 0/32768\n\x1b[INFO]  \x1bsnarkJS\x1b: Circuit Hash: \n\t\td144fa51 d7843885 a93b86

,template instances,non-linear constraints,linear constraints,public inputs,public outputs,private inputs,private outputs,wires,labels,length,phase_2_compile_time
0,291,17676,0,8,0,76,0,17750,56674,0,NaN
1,292,17680,0,8,0,76,0,17754,56688,1,97.26689


In [99]:
df['differences'] = df["non-linear constraints"].diff()
# df.to_csv(CIRCUITS_DIR/"set_membership_data.csv")
df['differences']

0         NaN
1    -14030.0
2       484.0
3       484.0
4       484.0
5       484.0
10     2420.0
11      484.0
12      484.0
13      484.0
14      484.0
15      484.0
16      484.0
17      484.0
18      484.0
19      484.0
20      484.0
25     2420.0
30     2420.0
50     9680.0
Name: differences, dtype: float64

In [100]:
df

,template instances,non-linear constraints,linear constraints,public inputs,public outputs,private inputs,private outputs,wires,labels,length,differences
0,291,17676,0,8,0,76,0,17750,56674,0,NaN
1,292,3646,0,9,0,18,0,3663,11689,1,-14030.0
2,292,4130,0,9,0,20,0,4149,13241,2,484.0
3,292,4614,0,9,0,22,0,4635,14793,3,484.0
4,292,5098,0,9,0,24,0,5121,16345,4,484.0
5,292,5582,0,9,0,26,0,5607,17897,5,484.0
10,292,8002,0,9,0,36,0,8037,25657,10,2420.0
11,292,8486,0,9,0,38,0,8523,27209,11,484.0
12,292,8970,0,9,0,40,0,9009,28761,12,484.0
13,292,9454,0,9,0,42,0,9495,30313,13,484.0


### Compilation Contraints - Conclusion

* Set membership adds 2 constraints per set-member added.

* Merkle-trees add 484 contraints per level. 

Considering $2^l = capacity$ of the merkle tree

\begin{equation}
$$#set_contraints = members*2 + 2$$

$$#merkle_tree_contraints = \log_2{members}*484 + constant$$
\end{equation}

solving for $#merkle_tree_constraints < #set_contraints$ we need around sets of size around $2^{10}$ to be worth switching to merkle-trees taking in consideration only the number of contraints

## Merkle trees

In [96]:
# shutil.copy("./circuits/test/poseidon_vanchor_2_2.circom", ANALYSIS_FILE) 

def change_merkle_tree_depth(filename: str, depth: int):
    with open(filename, "r") as file:
        file_lines = file.readlines()
        base_file = file_lines[:-1]
        relevant_line = file_lines[-1]
        if relevant_line.endswith('\n'):
            relevant_line = relevant_line[:-1]
        
    changed_line = relevant_line.split()
    changed_line[-5] = f"Transaction({depth},"
    changed_line = ' '.join(changed_line)
    print(changed_line)
    
    with open(filename, "w") as file:
        for line in base_file:
            write_line(file, line)
        write_line(file, changed_line)
#         file.write(f'{changed_line}\n')

change_merkle_tree_depth(ANALYSIS_FILE, 14)

component main {public [publicAmount, extDataHash, inputNullifier, outputCommitment, chainID, roots]} = Transaction(14, 2, 2, 11850551329423159860688778991827824730037759162201783566284850822760196767874, 2);


In [98]:
# shutil.copy("./circuits/test/poseidon_vanchor_2_2.circom", ANALYSIS_FILE) 
        
set_lengths = [1, 2, 3, 4, 5, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 25, 30, 50]
# set_lengths = [1]
df = get_baseline()
for set_length in set_lengths:
    change_merkle_tree_depth(ANALYSIS_FILE, set_length)
    start = time.time()
    df = pd.concat([df, get_compilation_data(ANALYSIS_FILE, set_length)])
#     generate_phase2(set_length)
    end = time.time()
    took = end-start
#     df.loc[df["length"]==set_length, "phase_2_compile_time"] = took
    print(f'TOOK {took:.2f}s TO COMPILE PHASE 2 FOR {set_length}')
    

df.head()

parsing compilation for length=0
component main {public [publicAmount, extDataHash, inputNullifier, outputCommitment, chainID, roots]} = Transaction(1, 2, 2, 11850551329423159860688778991827824730037759162201783566284850822760196767874, 2);
parsing compilation for length=1
TOOK 1.53s TO COMPILE PHASE 2 FOR 1
component main {public [publicAmount, extDataHash, inputNullifier, outputCommitment, chainID, roots]} = Transaction(2, 2, 2, 11850551329423159860688778991827824730037759162201783566284850822760196767874, 2);
parsing compilation for length=2
TOOK 1.56s TO COMPILE PHASE 2 FOR 2
component main {public [publicAmount, extDataHash, inputNullifier, outputCommitment, chainID, roots]} = Transaction(3, 2, 2, 11850551329423159860688778991827824730037759162201783566284850822760196767874, 2);
parsing compilation for length=3
TOOK 1.61s TO COMPILE PHASE 2 FOR 3
component main {public [publicAmount, extDataHash, inputNullifier, outputCommitment, chainID, roots]} = Transaction(4, 2, 2, 11850551329

,template instances,non-linear constraints,linear constraints,public inputs,public outputs,private inputs,private outputs,wires,labels,length
0,291,17676,0,8,0,76,0,17750,56674,0
1,292,3646,0,9,0,18,0,3663,11689,1
2,292,4130,0,9,0,20,0,4149,13241,2
3,292,4614,0,9,0,22,0,4635,14793,3
4,292,5098,0,9,0,24,0,5121,16345,4


# Clean up

In [43]:
try:
    os.remove('baseline.r1cs')
    os.remove('baseline.sym')
except FileNotFoundError:
    print('already cleaned up')
    
finally:
    ! rm -rf ./baseline_js

already cleaned up


# Conclusion

### As shown above, the setMembership circuit adds 2 contraints for being used and 2 contraints extra per member in the set